In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import json
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE, RMSE, SMAPE
import joblib
import warnings
warnings.filterwarnings("ignore")


In [ ]:

# Set plot style
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:

print("Loading data and model...")
# Load test data
test_data = pd.read_csv('../data/test_data.csv')
train_data = pd.read_csv('../data/train_data.csv')
val_data = pd.read_csv('../data/val_data.csv')


In [ ]:

# Load original data for reference
original_data = pd.read_csv('../sales_data.csv')


In [ ]:

# Load model metadata
with open('../models/model_metadata.json', 'r') as f:
    model_metadata = json.load(f)


In [ ]:

# Load scaler to reverse transformations
scaler = joblib.load('../models/feature_scaler.joblib')


In [ ]:

# Load feature config
with open('../data/feature_config.json', 'r') as f:
    feature_config = json.load(f)


In [ ]:

# Create test dataset with the same parameters as training
test_dataset = TimeSeriesDataSet(
    data=test_data,
    time_idx=model_metadata["time_idx"],
    target=model_metadata["target"],
    group_ids=model_metadata["group_ids"],
    max_encoder_length=model_metadata["max_encoder_length"],
    max_prediction_length=model_metadata["max_prediction_length"],
    static_categoricals=model_metadata["static_categoricals"],
    static_reals=model_metadata["static_reals"],
    time_varying_known_categoricals=model_metadata["time_varying_known_categoricals"],
    time_varying_known_reals=model_metadata["time_varying_known_reals"],
    time_varying_unknown_categoricals=model_metadata["time_varying_unknown_categoricals"],
    time_varying_unknown_reals=model_metadata["time_varying_unknown_reals"],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
)


In [ ]:

test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64)


In [ ]:

# Load best model
print("Loading trained model...")
try:
    best_model_path = "../models/checkpoints/tft-sales-forecasting-best.ckpt"
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
except FileNotFoundError:
    print("Using saved model state instead of checkpoint...")
    # Initialize model architecture
    training = TimeSeriesDataSet(
        data=train_data,
        time_idx=model_metadata["time_idx"],
        target=model_metadata["target"],
        group_ids=model_metadata["group_ids"],
        max_encoder_length=model_metadata["max_encoder_length"],
        max_prediction_length=model_metadata["max_prediction_length"],
        static_categoricals=model_metadata["static_categoricals"],
        static_reals=model_metadata["static_reals"],
        time_varying_known_categoricals=model_metadata["time_varying_known_categoricals"],
        time_varying_known_reals=model_metadata["time_varying_known_reals"],
        time_varying_unknown_categoricals=model_metadata["time_varying_unknown_categoricals"],
        time_varying_unknown_reals=model_metadata["time_varying_unknown_reals"],
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )
    best_tft = TemporalFusionTransformer.from_dataset(training)
    # Load saved weights
    best_tft.load_state_dict(torch.load('../models/tft_model.pth'))


In [ ]:

print("Making predictions...")
# Make predictions
predictions = best_tft.predict(test_dataloader, return_x=True, return_y=True)


In [ ]:

# Extract actual and predicted values
x, y = predictions.x, predictions.y
prediction_values = predictions.output.detach().cpu().numpy().flatten()
actual_values = y[0].detach().cpu().numpy().flatten()


In [ ]:

# Create DataFrame with predictions and actuals
results = pd.DataFrame({
    'actual': actual_values,
    'predicted': prediction_values
})


In [ ]:

# Inverse transform if needed (assuming sales was standardized)
# Extract column indices for numeric features
orig_numeric_features = ['prev_quarter_sales', 'total_quarter_sales', 'avg_quarterly_sales', 'sales']
scaled_cols_idx = {col: i for i, col in enumerate(orig_numeric_features)}
sales_idx = scaled_cols_idx.get('sales', 0)  # Default to 0 if not found


In [ ]:

# Create arrays to inverse transform
actual_scaled = np.zeros((len(results), len(orig_numeric_features)))
actual_scaled[:, sales_idx] = results['actual']

pred_scaled = np.zeros((len(results), len(orig_numeric_features)))
pred_scaled[:, sales_idx] = results['predicted']


In [ ]:

# Inverse transform
actual_unscaled = scaler.inverse_transform(actual_scaled)[:, sales_idx]
pred_unscaled = scaler.inverse_transform(pred_scaled)[:, sales_idx]


In [ ]:

# Add to results DataFrame
results['actual_unscaled'] = actual_unscaled
results['predicted_unscaled'] = pred_unscaled
results['prediction_error'] = results['actual_unscaled'] - results['predicted_unscaled']
results['percentage_error'] = (results['prediction_error'] / results['actual_unscaled']) * 100


In [ ]:

# Get test data indices
idx = list(x["input_ids"].detach().cpu().numpy().flatten())
test_indices = [i for i, val in enumerate(idx) if val != -100]


In [ ]:

# Add entity information
results['entity_id'] = test_data.iloc[test_indices]['entity_id'].values
results['time_idx'] = test_data.iloc[test_indices]['time_idx'].values
results['distributor_id'] = test_data.iloc[test_indices]['distributor_id'].values
results['sku'] = test_data.iloc[test_indices]['sku'].values
results['quarter'] = test_data.iloc[test_indices]['quarter'].values
results['year'] = test_data.iloc[test_indices]['year'].values


In [ ]:

# Calculate metrics
mae = MAE()(torch.tensor(results['predicted_unscaled'].values), torch.tensor(results['actual_unscaled'].values))
rmse = RMSE()(torch.tensor(results['predicted_unscaled'].values), torch.tensor(results['actual_unscaled'].values))
smape = SMAPE()(torch.tensor(results['predicted_unscaled'].values), torch.tensor(results['actual_unscaled'].values))
mape = np.mean(np.abs(results['percentage_error'].values))


In [ ]:

# Print metrics
print("\n--- Model Performance Metrics ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


In [ ]:

# Visualization of predictions vs actuals
plt.figure(figsize=(12, 8))
plt.scatter(results['actual_unscaled'], results['predicted_unscaled'], alpha=0.5)
plt.plot([min(results['actual_unscaled']), max(results['actual_unscaled'])], 
         [min(results['actual_unscaled']), max(results['actual_unscaled'])], 
         'r--')
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Predicted vs Actual Sales')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

# Distribution of prediction errors
plt.figure(figsize=(10, 6))
plt.hist(results['prediction_error'], bins=30, alpha=0.75)
plt.axvline(x=0, color='red', linestyle='--')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.grid(True)
plt.show()


In [ ]:

# Calculate MAPE by category
category_errors = results.join(test_data.iloc[test_indices][['category']], how='left')
category_mape = category_errors.groupby('category')['percentage_error'].apply(lambda x: np.mean(np.abs(x))).sort_values()


In [ ]:

plt.figure(figsize=(12, 7))
category_mape.plot(kind='bar')
plt.title('Mean Absolute Percentage Error by Product Category')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# Calculate MAPE by distributor
distributor_mape = category_errors.groupby('distributor_id')['percentage_error'].apply(lambda x: np.mean(np.abs(x))).sort_values()

plt.figure(figsize=(14, 8))
distributor_mape[:15].plot(kind='bar')
plt.title('Mean Absolute Percentage Error by Top 15 Distributors')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# Analyze festival impact on prediction accuracy
festival_columns = ['is_diwali', 'is_ganesh_chaturthi', 'is_gudi_padwa', 'is_eid', 
                   'is_akshay_tritiya', 'is_dussehra_navratri', 'is_onam', 'is_christmas']


In [ ]:

# Create extended results with festival data
extended_results = results.join(test_data.iloc[test_indices][festival_columns], how='left')


In [ ]:

# Calculate MAPE for festival and non-festival periods
extended_results['has_festival'] = extended_results[festival_columns].sum(axis=1) > 0

festival_mape = extended_results.groupby('has_festival')['percentage_error'].apply(lambda x: np.mean(np.abs(x)))
print("\n--- Festival Impact on Prediction Accuracy ---")
print(f"MAPE during festival periods: {festival_mape.get(True, float('nan')):.2f}%")
print(f"MAPE during non-festival periods: {festival_mape.get(False, float('nan')):.2f}%")


In [ ]:

# Create specific festival MAPE
festival_impact = {}
for festival in festival_columns:
    festival_rows = extended_results[extended_results[festival] == 1]
    if len(festival_rows) > 0:
        festival_impact[festival] = np.mean(np.abs(festival_rows['percentage_error']))
    else:
        festival_impact[festival] = float('nan')


In [ ]:

# Plot festival impact
plt.figure(figsize=(12, 6))
impact_df = pd.DataFrame(list(festival_impact.items()), columns=['Festival', 'MAPE'])
impact_df = impact_df.sort_values('MAPE')
sns.barplot(x='Festival', y='MAPE', data=impact_df)
plt.title('Prediction Error by Festival')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# Analyze model performance by movement category
movement_mape = extended_results.join(test_data.iloc[test_indices][['movement_category']], how='left')
movement_mape = movement_mape.groupby('movement_category')['percentage_error'].apply(lambda x: np.mean(np.abs(x)))

print("\n--- Prediction Accuracy by Movement Category ---")
for category, mape in movement_mape.items():
    print(f"MAPE for {category} items: {mape:.2f}%")


In [ ]:

plt.figure(figsize=(10, 6))
movement_mape.plot(kind='bar')
plt.title('Mean Absolute Percentage Error by Movement Category')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=0)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# Generate order recommendations
print("\n--- Sample Order Recommendations ---")


In [ ]:

# Sample 5 random entities
sample_entities = results['entity_id'].sample(5).unique()

for entity in sample_entities:
    entity_data = results[results['entity_id'] == entity]
    if len(entity_data) == 0:
        continue
        
    distributor = entity_data['distributor_id'].iloc[0]
    sku = entity_data['sku'].iloc[0]
    
    # Get historical data for context
    historical = original_data[(original_data['distributor_id'] == distributor) & (original_data['sku'] == sku)]
    
    # Get the most recent prediction
    recent = entity_data.sort_values('time_idx', ascending=False).iloc[0]
    
    # Calculate average historical sales for this distributor-SKU pair
    avg_sales = historical['sales'].mean()
    
    # Get category information
    try:
        category = historical['category'].iloc[0]
        movement = historical['movement_category'].iloc[0]
    except:
        category = "Unknown"
        movement = "Unknown"
    
    # Create recommendation
    predicted_sales = recent['predicted_unscaled']
    if predicted_sales < 0:
        predicted_sales = 0  # Ensure no negative predictions
        
    # Apply a safety buffer based on movement category
    buffer_factor = 1.2 if movement == "Fast Moving" else 1.1 if movement == "Medium" else 1.05
    recommendation = predicted_sales * buffer_factor
    
    print(f"\nDistributor: {distributor}, SKU: {sku}, Category: {category}, Movement: {movement}")
    print(f"Next quarter predicted sales: {predicted_sales:.2f}")
    print(f"Recommended order quantity: {recommendation:.2f}")
    print(f"Average historical sales: {avg_sales:.2f}")
    
    # Check if festival quarter
    quarter = recent['quarter']
    year = recent['year']
    festival_cols = [col for col in festival_columns if col in historical.columns]
    festival_quarter = historical[(historical['quarter'] == quarter) & (historical['year'] == year)][festival_cols].sum().sum() > 0
    
    if festival_quarter:
        print("Note: Festival quarter detected! Consider additional inventory.")


In [ ]:

# Save recommendations to CSV
print("\n--- Generating Full Recommendations ---")

# Create recommendations for all entities
recommendations = []

# Group by entity and find most recent prediction
for entity, group in results.groupby('entity_id'):
    recent = group.sort_values('time_idx', ascending=False).iloc[0]
    
    distributor = recent['distributor_id']
    sku = recent['sku']
    
    # Get historical data
    historical = original_data[(original_data['distributor_id'] == distributor) & (original_data['sku'] == sku)]
    
    # Get metadata
    try:
        category = historical['category'].iloc[0]
        movement = historical['movement_category'].iloc[0]
    except:
        category = "Unknown"
        movement = "Unknown"
    
    # Calculate predicted sales and recommendation
    predicted_sales = max(0, recent['predicted_unscaled'])  # Ensure no negative predictions
    
    # Apply buffer based on movement category
    buffer_factor = 1.2 if movement == "Fast Moving" else 1.1 if movement == "Medium" else 1.05
    recommendation = predicted_sales * buffer_factor
    
    # Check if festival quarter
    quarter = recent['quarter']
    year = recent['year']
    festival_cols = [col for col in festival_columns if col in historical.columns]
    festival_quarter = historical[(historical['quarter'] == quarter) & (historical['year'] == year)][festival_cols].sum().sum() > 0
    
    # Add festival boost if needed
    if festival_quarter:
        recommendation *= 1.15  # Add 15% for festivals
    
    # Store recommendation
    recommendations.append({
        'distributor_id': distributor,
        'sku': sku,
        'category': category,
        'movement_category': movement,
        'quarter': quarter,
        'year': year,
        'predicted_sales': predicted_sales,
        'recommended_order': recommendation,
        'has_festival': festival_quarter
    })


In [ ]:

# Convert to DataFrame and save
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.to_csv('../results/order_recommendations.csv', index=False)

print(f"Order recommendations generated for {len(recommendations_df)} distributor-SKU combinations")
print("Recommendations saved to '../results/order_recommendations.csv'")


In [ ]:

# Show feature importance from TFT model
print("\n--- Feature Importance Analysis ---")
interpretation = best_tft.interpret_output(test_dataloader)


In [ ]:

# Variable importance plot
plt.figure(figsize=(10, 8))
best_tft.plot_variable_importance(interpretation)
plt.title('TFT Model Feature Importance')
plt.tight_layout()
plt.show()

# Attention plot
plt.figure(figsize=(12, 8))
try:
    interpretation2 = best_tft.interpret_output(test_dataloader, attention=True)
    best_tft.plot_attention(interpretation2.attention)
    plt.title('TFT Attention Mechanism')
    plt.tight_layout()
    plt.show()
except:
    print("Unable to plot attention mechanism - may require retraining with additional parameters")

print("\n--- Key Findings and Recommendations ---")
print("1. The model achieves an overall MAPE of {:.2f}%, indicating good accuracy for inventory planning".format(mape))
print("2. Performance varies across product categories with best predictions for: " + 
      ", ".join(category_mape.nsmallest(3).index.tolist()))
print("3. Festival periods impact prediction accuracy - special consideration needed")
print("4. Different buffer strategies recommended based on movement category")
print("5. Order recommendations incorporate both prediction and category-specific safety factors")

In [ ]:

# Group by entity and find most recent prediction
for entity, group in results.groupby('entity_id'):
    recent = group.sort_values('time_idx', ascending=False).iloc[0]
    
    distributor = recent['distributor_id']
    sku = recent['sku']
    
    # Get historical data
    historical = original_data[(original_data['distributor_id'] == distributor) & (original_data['sku'] == sku)]
    
    # Get metadata
    try:
        category = historical['category'].iloc[0]
        movement = historical['movement_category'].iloc[0]
    except:
        category = "Unknown"
        movement = "Unknown"
    
    # Calculate predicted sales and recommendation
    predicted_sales = max(0, recent['predicted_unscaled'])  # Ensure no negative predictions
    
    # Apply buffer based on movement category
    buffer_factor = 1.2 if movement == "Fast Moving" else 1.1 if movement == "Medium" else 1.05
    recommendation = predicted_sales * buffer_factor
    
    # Check if festival quarter
    quarter = recent['quarter']
    year = recent['year']
    festival_cols = [col for col in festival_columns if col in historical.columns]
    festival_quarter = historical[(historical['quarter'] == quarter) & (historical['year'] == year)][festival_cols].sum().sum() > 0
    
    # Add festival boost if needed
    if festival_quarter:
        recommendation *= 1.15  # Add 15% for festivals
    
    # Store recommendation
    recommendations.append({
        'distributor_id': distributor,
        'sku': sku,
        'category': category,
        'movement_category': movement,
        'quarter': quarter,
        'year': year,
        'predicted_sales': predicted_sales,
        'recommended_order': recommendation,
        'has_festival': festival_quarter
    })


In [ ]:

# Convert to DataFrame and save
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.to_csv('../results/order_recommendations.csv', index=False)

print(f"Order recommendations generated for {len(recommendations_df)} distributor-SKU combinations")
print("Recommendations saved to '../results/order_recommendations.csv'")


In [ ]:

# Show feature importance from TFT model
print("\n--- Feature Importance Analysis ---")
interpretation = best_tft.interpret_output(test_dataloader)


In [ ]:

# Variable importance plot
plt.figure(figsize=(10, 8))
best_tft.plot_variable_importance(interpretation)
plt.title('TFT Model Feature Importance')
plt.tight_layout()
plt.show()


In [ ]:

# Attention plot
plt.figure(figsize=(12, 8))
try:
    interpretation2 = best_tft.interpret_output(test_dataloader, attention=True)
    best_tft.plot_attention(interpretation2.attention)
    plt.title('TFT Attention Mechanism')
    plt.tight_layout()
    plt.show()
except:
    print("Unable to plot attention mechanism - may require retraining with additional parameters")

print("\n--- Key Findings and Recommendations ---")
print("1. The model achieves an overall MAPE of {:.2f}%, indicating good accuracy for inventory planning".format(mape))
print("2. Performance varies across product categories with best predictions for: " + 
      ", ".join(category_mape.nsmallest(3).index.tolist()))
print("3. Festival periods impact prediction accuracy - special consideration needed")
print("4. Different buffer strategies recommended based on movement category")
print("5. Order recommendations incorporate both prediction and category-specific safety factors")